In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np
tz = pytz.timezone('America/Edmonton')

In [11]:
def release_token(accessToken):
    path = '/api/ReleaseToken'
    server = 'api.nrgstream.com'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)

release_token(default_pickle['accessToken'])

In [12]:
t = datetime.now(pytz.timezone('America/Edmonton')).date() - timedelta(days=8)
t2 = datetime.now(pytz.timezone('America/Edmonton')) - timedelta(days=8)
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (t2,monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (t2,daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle

{'accessToken': 'ZPTmuJIcJ2DfaHDzGfhZxum70SOlii8RAQmcy7c42twiTwky49_Ej_3P45f2bvUHYfT8aLBbpppffZn5DAiYl1pWsnvxlTybR79bi21ug5kwac2I913QqXJrsnOWvZL25uUyu0VKLyKu9mKkHqbO34IzbnfzTKR8HXqLs72v4UvgEGcIrG06QTvT0XOn8sItX8KGFyCBHNxxB3hmCVi1ybejCq8k5bUNwMU_FVl7aoogO4RNGlRmdulOqtnvgg2xGmzxJMSiWVKkeZR-tFaQn8twCqMr9uezvXwG1pF4zaFvFUBQAaP7b7AzrtyUhFuPpKp_BAQlQfMJf-V4wFkMNV7guY8',
 'current_data': (datetime.datetime(2022, 9, 22, 11, 58, 33, 702641, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                      timeStamp    value    fuelType
  0   2022-09-22 00:00:00-06:00  1265.00        Coal
  1   2022-09-22 00:05:00-06:00  1267.00        Coal
  2   2022-09-22 00:10:00-06:00  1264.00        Coal
  3   2022-09-22 00:15:00-06:00  1262.00        Coal
  4   2022-09-22 00:20:00-06:00  1261.00        Coal
  ..                        ...      ...         ...
  139 2022-09-22 11:35:00-06:00    89.08  Pool Price
  140 2022-09-22 11:40:00-06:00    89.08  Pool Price
  141 2022-09-22 11:45:

In [11]:
default_pickle['daily_outage_dfs']

[(datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                     timeStamp  value  fuelType
  0  2022-09-12 00:00:00-06:00    750  Intertie
  1  2022-09-13 00:00:00-06:00      0  Intertie
  2  2022-09-14 00:00:00-06:00      0  Intertie
  3  2022-09-15 00:00:00-06:00      0  Intertie
  4  2022-09-16 00:00:00-06:00      0  Intertie
  ..                       ...    ...       ...
  86 2022-12-07 00:00:00-07:00      0      Wind
  87 2022-12-08 00:00:00-07:00      0      Wind
  88 2022-12-09 00:00:00-07:00      0      Wind
  89 2022-12-10 00:00:00-07:00      0      Wind
  90 2022-12-11 00:00:00-07:00      0      Wind
  
  [819 rows x 3 columns]),
 (datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                     timeStamp  value  fuelType
  0  2022-09-12 00:00:00-06:00    750  Intertie
  1  2022-09-13 00:00:00-06:00      0  Intertie
  2  2022-09-14 00

In [5]:
default_pickle['daily_outage_dfs'][6][1][default_pickle['daily_outage_dfs'][6][1]['fuelType'] != ('3-Day Solar Forecast' or '7-Day Wind Forecast')]

,timeStamp,value,fuelType
0,2022-09-19 00:00:00-06:00,543.00,Intertie
1,2022-09-19 01:00:00-06:00,0.00,Intertie
2,2022-09-19 02:00:00-06:00,0.00,Intertie
3,2022-09-19 03:00:00-06:00,0.00,Intertie
4,2022-09-19 04:00:00-06:00,0.00,Intertie
...,...,...,...
163,2022-09-25 19:00:00-06:00,240.12,7-Day Wind Forecast
164,2022-09-25 20:00:00-06:00,282.14,7-Day Wind Forecast
165,2022-09-25 21:00:00-06:00,334.81,7-Day Wind Forecast
166,2022-09-25 22:00:00-06:00,390.90,7-Day Wind Forecast


In [5]:
with open('./temp.pickle', 'rb') as handle:
    accessToken = pickle.load(handle)
accessToken


'HMgsNGXA7rzhEsJbUPmK_XqzIIbP4ZpA6Rri6yFOeKdO4jp5Ya3_7bDbf5y9-jM8O8hEi3xt-h1Eo5aiRBULKm2ca1qdEs8CsYQQPrlnLxeLn5KuvLphWuRVDwp8RNWlV8DdB_xGcgwa7AVb9_YPMRQSaSzq_9E7_Nypznbyn2Q77XYMtwFQ0IrDPGkHULODtMhkKnjS6Nk3aWRDxpRAs2QYgv0jeiIpAjdehc1vcSXMIhic6KFCF64xY3bA1G5BVPsIE3nP0E6Zzbi-NlWfloO46X-ebFqoRz11NDVdxaSwKbjMk_BSsZbjP3Q2BW3GnrJ8vGF08FPDxQFu1Q_iy6wkVEo'

In [17]:
dct = {'coal':10,'gas':20,'wind':30,'solar':40}
dct2 = {'coal','gas'}
{dct2:dct[dct2] for dct2 in dct2}

{'coal': 10, 'gas': 20}